<a href="https://colab.research.google.com/github/samibahig/recover/blob/master/RandomSCM_M%C3%A9tabolomique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl
#print('\nMETADATA :')
data_path = ''
metadata_filename = '/content/metadata.csv'
meta_df = pd.read_csv(metadata_filename)
print('-----------')
print(meta_df)
print('-----------')
#print(meta_df.columns)
meta_df.columns = ['#', 'plate', '-', 'symptoms'] + list(meta_df)[4:]
#print(meta_df.columns)
#print('available metadata :', list(meta_df))
meta_idx = meta_df['ID'].to_list()
meta_label = meta_df['symptoms'].to_list()
#print('------------------')
#print(list(zip(meta_idx, meta_label)))
#print('------------------')
meta_id_label_dict = {str(k): 1 if v=='S' else 0 for k, v in zip(meta_idx, meta_label)}


print('\nMETABOLOMICS DATA :')
metabolomics_data_filename = '/content/metabolomics.csv'
#print('data_path', '/content/metabolomics.csv')
feat_df = pd.read_csv(metabolomics_data_filename, index_col=0, skiprows=[0], dtype=str, usecols=[0])
features = list(feat_df.index.values)
print('# of features : ', len(features))
print('first feature :', features[0])
print('last feature :', features[-1])
col_list = ["Group"]
column1 = pd.read_csv(metabolomics_data_filename, usecols=col_list)
print('----------')
column1 = column1['Group'].to_list()
print(column1[1:])

print('----------')
idx_df = pd.read_csv(metabolomics_data_filename, header=1, nrows=1, encoding='unicode_escape')

idx = list(idx_df)[1:]
idx = [l[17:22] for l in idx]
labels_df = pd.read_csv(metabolomics_data_filename, nrows=1)
labels = list(labels_df)[1:]
print('# of labels : ', len(labels))

cols_df = pd.read_csv(metabolomics_data_filename, header=1, nrows=1, encoding='unicode_escape')
cols_list = list(cols_df)

df3 = pd.read_csv(metabolomics_data_filename, header=1, na_values=['#DIV/0!'], usecols=cols_list[1:], encoding='unicode_escape')
df3 = df3.T
df3['idx'] = idx
df3.set_index('idx', inplace=True)
df3.columns = features
df3 = df3.dropna(axis=1)

#clean data of samples that are not in metadata :
idx = df3.index.values
y = []
for k in range(len(idx)):
    id = idx[k]
    if id in meta_id_label_dict:
        y.append(meta_id_label_dict[id])
    else:
        # we will not put this sample in the dataset
        #print('sample to remove because of unknown label:', k, id)
        y.append('to_remove')
df3['label'] = y
df3 = df3[df3.label != 'to_remove']

#create X and y matrices for ML :
y = list(df3['label'])
print(y)
del df3['label']
X = df3.to_numpy()

print('metabolomics data :')
print('# of samples : ', df3.shape[0])
print('# of features : ', df3.shape[1])
print('labels:', list(dict.fromkeys(y)))

## to concatenate the 2 proteomics dataframes and the metabolomics if you want :
##df = pd.concat([df_1_2, df3], axis=1)
##df = df.dropna(axis=0)
##print('multi-omics df :')
##print('# of samples : ', df.shape[0])
##print('# of features : ', df.shape[1])

from sklearn.base import ClassifierMixin
from sklearn.ensemble import BaseEnsemble
from pyscm import SetCoveringMachineClassifier
from pyscm.rules import DecisionStump

from sklearn.utils.validation import check_X_y, check_array, check_is_fitted, check_random_state
from sklearn.utils.random import sample_without_replacement
from sklearn.metrics import accuracy_score
import numbers
import itertools
import numpy as np
from warnings import warn
from joblib import effective_n_jobs, Parallel, delayed

MAX_INT = np.iinfo(np.int32).max
def _parallel_build_estimators(idx, ensemble, p_of_estims, seeds, X, y, tiebreaker):
    """
    Fit SCM estimators on subsamples of the training data
    """
    estimators = []
    estim_features = []
    for k in idx:
        p_param = p_of_estims[k] # p param for the classifier to fit
        random_state = seeds[k]
        estim = SetCoveringMachineClassifier(p=p_param, max_rules=ensemble.max_rules, model_type=ensemble.model_type, random_state=random_state)
        feature_indices = sample_without_replacement(ensemble._pop_features, ensemble._max_features, random_state=random_state)
        samples_indices = sample_without_replacement(ensemble._pop_samples, ensemble._max_samples, random_state=random_state)
        Xk = (X[samples_indices])[:, feature_indices]
        yk = y[samples_indices]
        if len(list(set(yk))) < 2:
            raise ValueError("One of the subsamples contains elements from only 1 class, try increase max_samples value")
        if tiebreaker is None:
            estim.fit(Xk, yk)
        else:
            estim.fit(Xk, yk, tiebreaker)
        estim_features.append(feature_indices)
        estimators.append(estim)
    return estimators, estim_features


def _parallel_predict_proba(ensemble, X, idx, results):
    """
    Compute predictions of SCM estimators
    """
    for k in idx:
        res = ensemble.estimators[k].predict(X[:, ensemble.estim_features[k]])
        if ensemble.min_cq_combination:
            res[res==0] = -1
            results = results + ensemble.min_cq_weights[k]*res
        else:
            results = results + res
    return results


def _partition_estimators(n_estimators, n_jobs):
    """Private function used to partition estimators between jobs."""
    # Compute the number of jobs
    n_jobs = min(effective_n_jobs(n_jobs), n_estimators)

    # Partition estimators between jobs
    n_estimators_per_job = np.full(n_jobs, n_estimators // n_jobs,
                                   dtype=np.int)
    n_estimators_per_job[:n_estimators % n_jobs] += 1
    starts = np.cumsum(n_estimators_per_job)

    return n_jobs, n_estimators_per_job.tolist(), [0] + starts.tolist()


class RandomScmClassifier(BaseEnsemble, ClassifierMixin):
    """A Bagging classifier for SetCoveringMachineClassifier()
    The base estimators are built on subsets of both samples
    and features.
    Parameters
    ----------
    n_estimators : int, default=10
        The number of base estimators in the ensemble.
    max_samples : int or float, default=1.0
        The number of samples to draw from X to train each base estimator with
        replacement.
        - If int, then draw `max_samples` samples.
        - If float, then draw `max_samples * X.shape[0]` samples.
    max_features : int or float, default=1.0
        The number of features to draw from X to train each base estimator (
        without replacement.
        - If int, then draw `max_features` features.
        - If float, then draw `max_features * X.shape[1]` features.
    max_rules : int
        maximal number of rules for the scm estimators
    p_options : list of float with len =< n_estimators, default=[1.0]
        The estimators will be fitted with values of p found in p_options
        let k be k = n_estimators/len(p_options),
        the k first estimators will have p=p_options[0],
        the next k estimators will have p=p_options[1] and so on...
    model_type : string, default='conjunction'
        type of estimators to build
        accepted values : 'conjunction', 'disjunction'
    n_jobs : int, default=None
        The number of jobs to run in parallel for both fit and
        predict. 'None' means 1. '-1' means using all
        processors.
    random_state : int or RandomState, default=None
        Controls the random resampling of the original dataset
        (sample wise and feature wise).
        If the base estimator accepts a `random_state` attribute, a different
        seed is generated for each instance in the ensemble.
        Pass an int for reproducible output across multiple function calls.
        See :term:`Glossary <random_state>`.

    Attributes
    ----------
    n_features_ : int
        The number of features when :meth:`fit` is performed.
    estimators_ : list of estimators
        The collection of fitted base estimators.
    estim_features : list of arrays
        The subset of drawn features for each base estimator.

    Examples
    --------
    >>> random_scm = RandomScmClassifier(p_options=[2, 4], max_samples=0.5, max_features = 0.7)
    >>> random_scm.fit(X_train, y_train)
    >>> hyperparams = random_scm.get_hyperparams()
    >>> importances = random_scm.features_importance()
    >>> disagree = random_scm.classifiers_disagreement(X)

    References
    ----------
    .. [1] L. Breiman, "Pasting small votes for classification in large
           databases and on-line", Machine Learning, 36(1), 85-103, 1999.
    .. [2] G. Louppe and P. Geurts, "Ensembles on Random Patches", Machine
           Learning and Knowledge Discovery in Databases, 346-361, 2012.
    """
    def __init__(self,
                 n_estimators=100,
                 max_samples=0.5,
                 max_features=0.5,
                 max_rules=10,
                 p_options=[1.0],
                 model_type="conjunction",
                 n_jobs=None,
                 min_cq_combination=False,
                 min_cq_mu = 10e-3,
                 random_state=None):
        self.n_estimators = n_estimators
        self.max_samples = max_samples
        self.max_features = max_features
        self.max_rules = max_rules
        self.p_options = p_options
        self.model_type = model_type
        self.n_jobs = n_jobs
        self.min_cq_combination = min_cq_combination
        self.min_cq_mu = min_cq_mu
        self.random_state = random_state
        self.labels_to_binary = {}
        self.binary_to_labels = {}

    def p_for_estimators(self):
        """Return the value of p for each estimator to fit."""
        options_len = len(self.p_options) # number of options
        estims_with_same_p = self.n_estimators // options_len # nb of estimators to fit with the same p
        p_of_estims = []
        if options_len > 1:
            for k in range(options_len - 1):
                opt = self.p_options[k] # an option
                p_of_estims = p_of_estims + ([opt] * estims_with_same_p) # estims_with_same_p estimators with p=opt
        p_of_estims = p_of_estims + ([self.p_options[-1]] * (self.n_estimators - len(p_of_estims)))
        return p_of_estims

    def get_estimators(self):
        """Return the list of estimators of the classifier"""
        if hasattr(self, 'estimators'):
            return self.estimators
        else:
            return "not defined (model not fitted)"

    def get_hyperparams(self):
        """Return the model hyperparameters"""
        hyperparams = {
            'n_estimators' : self.n_estimators, 
            'max_samples' : self.max_samples, 
            'max_features' : self.max_features, 
            'max_rules' : self.max_rules, 
            'p_options' : self.p_options, 
            'model_type' : self.model_type, 
            'random_state' : self.random_state
        }
        return hyperparams

    def labels_conversion(self, labels_list):
        """
        Return the equivalence between labels and binaries
        """
        l = list(set(labels_list))
        labels_dict = {c:idx for idx, c in enumerate(l)}
        if len(l) < 2:
            raise ValueError("Only 1 classe given to the model, needs 2")
        elif len(l) > 2:
             raise ValueError("{} classes were given, multiclass prediction is not implemented".format(len(l)))
        return np.array(l), labels_dict


    def fit(self, X, y, get_feature_importances=True, tiebreaker=None):
        """
        Fit the model with the given data
        """
        # Check if 2 classes are inputed and convert labels to binary labels
        X, y = check_X_y(X, y)
        self.classes_, self.labels_to_binary = self.labels_conversion(y)
        self.binary_to_labels = {bin_label:str_label for str_label, bin_label in self.labels_to_binary.items()}
        y = np.array([self.labels_to_binary[l] for l in y])

        # Save the original number of features
        self.n_features = X.shape[1]

        self.estimators = []
        self.estim_features = []
        max_rules = self.max_rules
        p_of_estims_ = self.p_for_estimators()
        model_type = self.model_type

        #seeds for reproductibility
        random_state = self.random_state
        random_state = check_random_state(random_state)
        seeds = random_state.randint(MAX_INT, size=self.n_estimators)
        self._seeds = seeds

        pop_samples, pop_features = X.shape
        max_samples, max_features = self.max_samples, self.max_features

        # validate max_samples
        if not isinstance(max_samples, numbers.Integral):
            max_samples = int(max_samples * pop_samples)
        if not (0 < max_samples <= pop_samples):
            raise ValueError("max_samples must be in (0, n_samples)")
        # store validated integer row sampling values
        self._max_samples = max_samples
        self._pop_samples = pop_samples

        # validate max_features
        if isinstance(self.max_features, numbers.Integral):
            max_features = self.max_features
        elif isinstance(self.max_features, np.float):
            max_features = self.max_features * pop_features
        else:
            raise ValueError("max_features must be int or float")
        if not (0 < max_features <= pop_features):
            raise ValueError("max_features must be in (0, n_features)")
        max_features = max(1, int(max_features))
        # store validated integer feature sampling values
        self._max_features = max_features
        self._pop_features = pop_features

        # parallel loop
        n_jobs, n_estimators_list, starts = _partition_estimators(self.n_estimators, self.n_jobs)
        # building estimators
        all_results = Parallel(n_jobs=n_jobs)(delayed(_parallel_build_estimators)(
                range(starts[i],starts[i+1]), self, p_of_estims_, seeds, X, y, tiebreaker)
            for i in range(n_jobs))

        self.estimators += list(itertools.chain.from_iterable(
            t[0] for t in all_results))
        self.estim_features += list(itertools.chain.from_iterable(
            t[1] for t in all_results))

        if self.min_cq_combination:
            predictions = np.zeros((X.shape[0], self.n_estimators))
            for k in range(self.n_estimators):
                predictions[:, k] = self.estimators[k].predict(X[:, self.estim_features[k]])

            from .min_cq import MinCqLearner
            mincq = MinCqLearner(self.min_cq_mu, "stumps", n_stumps_per_attribute=1,
                                 self_complemented=False)
            mincq.fit(predictions, y)
            self.min_cq_weights = mincq.majority_vote.weights
            self.min_cq_weights /= np.sum(np.abs(self.min_cq_weights))

        if get_feature_importances:
            importances = self.features_importance()
            self.feature_importances_ = np.array([importances['avg'][k]
                                                  if k in importances['avg'] else 0
                                                  for k in range(self.n_features)])
            self.feature_importances_ /= np.sum(self.feature_importances_)

    def predict(self, X):
        """
        Compute model predictions for data in X

        Returns:
        ----------
        predictions : array
            predictions[i] is the predicted class for he sample i in X
        """
        check_is_fitted(self, ["estimators", "estim_features"])
        X = check_array(X)
        predicted_proba = self.predict_proba(X)
        predictions = np.array(np.argmax(predicted_proba, axis=1), dtype=int)
        predictions = np.array([self.binary_to_labels[l] for l in predictions])
        return predictions

    def predict_proba(self, X):
        """
        Predict class probabilities according to the model estimators

        Parameters :
        ----------
        X : array
            a dataset to predict

        Returns : 
        ----------
        proba : array
            proba[c] contains the estimated probability for each sample of X to belong to class c
        """
        check_is_fitted(self, ["estimators", "estim_features"])
        X = check_array(X)
        # parallel loop
        n_jobs, n_estimators_list, starts = _partition_estimators(self.n_estimators, self.n_jobs)
        results = np.zeros(X.shape[0])
        results = Parallel(n_jobs=n_jobs)(delayed(_parallel_predict_proba)(
                self, X, range(starts[i], starts[i+1]), results)
                for i in range(n_jobs))
        if not self.min_cq_combination:
            votes = sum(results) / self.n_estimators
            proba = np.array([np.array([1 - vote, vote]) for vote in votes])
        else:
            votes = sum(results)
            proba = np.array([np.array([(1 - vote)/2, (1 + vote)/2]) for vote in votes])
        return proba

    def features_importance(self):
        """
        Compute features importances in estimators rules

        Returns:
        ----------
        importance : dict
            importances['avg'] : dict (feature id as key, mean importance as value)
                The mean importance of each feature over the estimators.
            importances['max'] : dict (feature id as key, max importance as value)
                The maximal importance of each feature over the estimators.
        """
        check_is_fitted(self, ["estimators", "estim_features"])
        importances = {'avg' : {}, 'max' : {}} # average and maximal feature/rule importances
        feature_id_occurences = {} # number of occurences of a feature in subsamples
        for (estim, features_idx) in zip(self.estimators, self.estim_features):
            # increment the total occurences of the feature :
            for id_feat in features_idx:
                if id_feat in feature_id_occurences:
                    feature_id_occurences[id_feat] += 1
                else:
                    feature_id_occurences[id_feat] = 1
            # sum the rules importances :
            #rules_importances = estim.get_rules_importances() # activate it when pyscm will implement importance
            rules_importances = np.ones(len(estim.model_.rules)) #delete it when pyscm will implement importance
            for rule, importance in zip(estim.model_.rules, rules_importances):
                global_feat_id = features_idx[rule.feature_idx]
                if global_feat_id in importances['avg']:
                    importances['avg'][global_feat_id] += importance
                    if importance > importances['max'][global_feat_id]:
                        importances['max'][global_feat_id] = importance
                else:
                    importances['avg'][global_feat_id] = importance
                    importances['max'][global_feat_id] = importance
        importances['avg'] = {k: round(v / feature_id_occurences[k], 3) for k, v in importances['avg'].items()}
        return importances

    def all_data_tiebreaker(self, model_type, feature_idx, thresholds, rule_type, X, y):
        """
        Choose a rule between rule with equal utility
        Select the one that have the best accuracy if applied alone on all the data

        Parameters :
        ----------
        model_type : strint ('conjunction' or 'dijunction')
            type of the model
        feature_idx, thresholds, rule_type : arrays
            description of the rules
        X, y : arrays
            a dataset used to compare rules

        Returns:
        ----------
        ID of the rule to select
        """
        keep_id = 0
        keep_id_score = -1
        for k in range(len(feature_idx)):
            feat_id, threshold, r_type = feature_idx[k], thresholds[k], rule_type[k]
            stump = DecisionStump(feature_idx=feat_id, threshold=threshold, kind=r_type)
            rule_classif = stump.classify(X).astype('int')
            rule_global_score = (rule_classif == y).sum()
            if rule_global_score > keep_id_score:
                keep_id = k
                keep_id_score = rule_global_score
        return keep_id

    def classifiers_disagreement(self, X):
        """
        Compute disagreement between estimators

        Returns:
        ----------
        disagreement : dict 
            disagreement['global'] : int
                global disagreement, computed with the following formula :
                average(disagreement(c1, c2))
                for (c1, c2) all the pairs of distinct estimators in the model
                with 
                disagreement(c1, c2) = (# of examples where c1 and c2 differs)/(# of examples)
            disagreement['estims'] : list
                list of disagreements of each estimator with the global model
            disagreement['heatmap'] : matrix
                heatmap of mutual disagreement for each pair of classifiers
        """
        check_is_fitted(self, ["estimators", "estim_features"])
        disagreement = {'global' : 0, 'estims' : []}
        total_comp = 0 # number of comparison of estimators
        global_pred = self.predict(X)
        heatmap = np.zeros((self.n_estimators, self.n_estimators))
        for kA in range(len(self.estimators)):
            estimA = self.estimators[kA]
            predA = estimA.predict(X[:, self.estim_features[kA]])
            disagree = np.not_equal(global_pred, predA).sum()/len(X) # disagreement of estimator A and global model
            disagreement['estims'].append(disagree)
            for kB in range(len(self.estimators)):
                if kB == kA:
                    continue # do not compare the estimator with itself
                estimB = self.estimators[kB]
                predB = estimB.predict(X[:, self.estim_features[kB]])
                disagree = np.not_equal(predB, predA).sum()/len(X) # disagreement of estimators A and B
                heatmap[kA][kB] = disagree
                disagreement['global'] += disagree
                total_comp += 1
        disagreement['global'] = disagreement['global']/total_comp
        disagreement['heatmap'] = heatmap
        return disagreement
    
    def get_estimators_indices(self):
        """
        Get drawn indices along both sample and feature axes
        """
        for seed in self._seeds:
            # operations accessing random_state must be performed identically to those in 'fit'
            feature_indices = sample_without_replacement(self._pop_features, self._max_features, random_state=seed)
            samples_indices = sample_without_replacement(self._pop_samples, self._max_samples, random_state=seed)
            yield samples_indices

    def score(self, X, y):
        check_is_fitted(self, ["estimators", "estim_features"])
        X, y = check_X_y(X, y)
        return accuracy_score(y, self.predict(X))

-----------
    Unnamed: 0 Unnamed: 1  ... addition                                specify.1
0            1    Plate 1  ...      NaN                                      NaN
1            2    Plate 1  ...      5.0                                   Stress
2            3    Plate 1  ...      6.0  back pain is worse, shortness of breath
3            4    Plate 1  ...      2.0  Impression of a foreingh body in throat
4            5    Plate 1  ...      2.0                                      NaN
..         ...        ...  ...      ...                                      ...
95         100    Plate 2  ...      NaN                                      NaN
96         101    Plate 2  ...      NaN                                      NaN
97         102    Plate 2  ...      NaN                                      NaN
98         103    Plate 2  ...      NaN                                      NaN
99         104    Plate 2  ...      NaN                                      NaN

[100 rows x 46 

In [ ]:
pip install pyscm-ml

In [ ]:
result = RandomScmClassifier()

In [ ]:
print(X, y)

[[0.0 0.0 0.0 ... 1084.074486 91004.26917 -145.43122140000003]
 [0.0 0.0 0.0 ... 1106.098664 100736.478 -170.776811]
 [0.0 0.0 0.0 ... 1122.8278990000001 94610.99892 -140.8011661]
 ...
 [0.0 0.0 0.0 ... 1024.050104 70873.17068 218.3647556]
 [0.0 0.0 0.0 ... 1107.663733 72563.38294 234.9694451]
 [0.0 0.0 0.0 ... 1049.796854 72014.0392 216.1274312]] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
print(X_train, y_train)

[[0.0 0.0 0.0 ... 1169.085267 71840.17199 223.142849]
 [0.0 0.0 0.0 ... 1087.592648 76479.99123 534.2568309999999]
 [0.0 0.0 0.0 ... 1161.018205 73361.0715 301.5213309]
 ...
 [0.0 0.0 0.0 ... 1049.796854 72014.0392 216.1274312]
 [0.0 0.0 0.0 ... 1135.444097 79730.1114 -485.48692530000005]
 [0.0 0.0 0.0 ... 1071.039711 88879.55426 -170.47225490000002]] [0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1]


In [ ]:
from sklearn.model_selection import train_test_split 
from sklearn import metrics

In [ ]:
#### Bootstrapping ####
########################################################
# Creating empty list to hold accuracy values
AccuracyValues=[]
n_times=60

In [ ]:
## Performing bootstrapping
from sklearn import metrics
for i in range(n_times):
    #Split the data into training and testing set
    from sklearn.model_selection import train_test_split
    # Changing the seed value for each iteration
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42+i)
    result = RandomScmClassifier()
    result.fit(X_train, y_train)
    ########################################################
    prediction = result.predict(X_test)
    Accuracy=metrics.accuracy_score(y_test, prediction)
    print(Accuracy)
    AccuracyValues.append((Accuracy))
    #print(Accuracy)
    print(AccuracyValues)

1.0
[1.0]
1.0
[1.0, 1.0]
1.0
[1.0, 1.0, 1.0]
1.0
[1.0, 1.0, 1.0, 1.0]
1.0
[1.0, 1.0, 1.0, 1.0, 1.0]
1.0
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
1.0
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
1.0
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
0.9
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9]
1.0
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0]
1.0
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0]
1.0
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0]
1.0
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0]
1.0
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0]
1.0
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
1.0
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
0.9
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9]
1.0
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0]
1.0
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 

In [ ]:
###### Single Decision Tree Regression in Python #######
    #choose from different tunable hyper parameters
    #RegModel = tree.DecisionTreeRegressor(max_depth=3,criterion='mse')
 
    #Creating the model on Training Data
    #DTree=RegModel.fit(X_train,y_train)
    #prediction=DTree.predict(X_test)
 
    #Measuring accuracy on Testing Data
#Accuracy=100- (np.mean(np.abs((y_test - prediction) / y_test)) * 100)
    
    # Storing accuracy values
#AccuracyValues.append(np.round(Accuracy))
    
################################################
# Result of all bootstrapping trials
print(AccuracyValues)
 
# Final accuracy
print('Final average accuracy',np.mean(AccuracyValues))
#print("Test Accuracy:", metrics.accuracy_score(y_test, y_final ))

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 0.9, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 0.9, 1.0, 1.0, 0.9, 0.9, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 0.9, 1.0, 1.0, 1.0, 0.9]
Final average accuracy 0.9783333333333333


In [ ]:
print(prediction)
print(prediction.astype(int).sum())
print(len(y_test))
#(prediction).astype(int).sum()/len(y_test)

[1 1 1 0 0 1 0 1 1 1]
7
10


In [ ]:
result.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn import metrics
y_final = result.predict(X_test)
#result.accuracy_score(y_test, y_final )
print("Test Accuracy:", metrics.accuracy_score(y_test, y_final ))

Test Accuracy: 0.9
